In [135]:
import pandas as pd
import requests
from plyer import notification
import sqlite3

In [136]:
# URL base da API
base_url = "https://brasilapi.com.br/api/"

In [137]:
# Função para exibir um alerta de erro usando Plyer
def alerta(mensagem):
    notification.notify(
        title= "Erro na API BrasilAPI",
        message= mensagem,
        app_name= "Projeto Final",
    )

In [138]:
# Função para buscar dados da API
def buscar_dados(endpoint):
    url = f"{base_url}{endpoint}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        mensagem = f"Erro ao buscar dados da API. Código de status: {response.status_code}"
        alerta(mensagem)  # Chama a função de alerta
        return None

In [139]:
#Criado a conexão ao banco de dados.
conn = sqlite3.connect('ProjetoFinalDB.db')
cursor = conn.cursor()

In [140]:
# Tabela 1: Lista de Taxas do Brasil
cursor.execute('''CREATE TABLE IF NOT EXISTS Taxas (
                    Nome TEXT,
                    Valor NUMERIC
                )''')

# Tabela 2: Lista de Pix do Brasil
cursor.execute('''CREATE TABLE IF NOT EXISTS Pixs (
                    ISPB NUMERIC,
                    Nome TEXT,
                    NomeReduzido TEXT,
                    Modalidade TEXT,
                    Tipo TEXT
                )''')

# Tabela 2: Lista dos CPTECs
cursor.execute('''CREATE TABLE IF NOT EXISTS CPTECS (
                    Nome TEXT,
                    Estado TEXT,
                    Id NUMERIC
                )''')



In [141]:
# Tabela 1: Lista de Taxas do Brasil
taxas = buscar_dados("taxas/v1")
if taxas:
    
    # Criar um DataFrame com os atributos
    nome = [taxa['nome'] for taxa in taxas]
    valor = [taxa['valor'] for taxa in taxas]

    df_taxas = pd.DataFrame({
        "Nome": nome,
        "Valor": valor,
    })
    
    df_taxas['Valor'] = df_taxas['Valor'].astype(float) #Conversão de tipo de coluna
    df_taxas['Nome'] = df_taxas['Nome'].astype(str)
    df_taxas.fillna(0, inplace=True) #Tratamento de valores ausentes
    
    print(df_taxas.head(5))  # Exibe as 5 primeiras linhas do DataFrame

    Nome  Valor
0  Selic  12.75
1    CDI  12.65
2   IPCA   4.61


In [142]:
# Tabela 2: Lista de PIX do Brasil
pixs = buscar_dados("pix/v1/participants")
if pixs:

    # Criar um DataFrame com os atributos
    ispb = [pix['ispb'] for pix in pixs]
    nome = [pix['nome'] for pix in pixs]
    nome_reduzido = [pix['nome_reduzido'] for pix in pixs]
    modalidade_participacao = [pix['modalidade_participacao'] for pix in pixs]
    tipo_participacao = [pix['tipo_participacao'] for pix in pixs]

    df_pixs = pd.DataFrame({
        "ISPB": ispb,
        "Nome": nome,
        "NomeReduzido": nome_reduzido,
        "Modalidade": modalidade_participacao,
        "Tipo": tipo_participacao
    })
    
    df_pixs['ISPB'] = df_pixs['ISPB'].astype(int) #Conversão de tipo de coluna
    df_pixs['Nome'] = df_pixs['Nome'].astype(str) 
    df_pixs['NomeReduzido'] = df_pixs['NomeReduzido'].astype(str) 
    df_pixs['Modalidade'] = df_pixs['Modalidade'].astype(str)
    df_pixs['Tipo'] = df_pixs['Tipo'].astype(str)
    df_pixs.fillna(0, inplace=True) #Tratamento de valores ausentes
    df_pixs = df_pixs[df_pixs['Tipo'] == 'IDRT'] # Aplicar o filtro para selecionar apenas 'IDRT'

    print(df_pixs.head(5)) # Exibe as 5 primeiras linhas do DataFrame

     ISPB                                               Nome  \
2   68987  COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...   
3   75847  COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...   
4  106180  CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...   
6  259231  SICOOB UNIMAIS METROPOLITANA - COOPERATIVA DE ...   
7  309024  CENTRAL DAS COOPERATIVAS DE ECONOMIA E CREDITO...   

                                      NomeReduzido Modalidade  Tipo  
2                                CC ARACREDI LTDA.       PDCT  IDRT  
3  CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL       PDCT  IDRT  
4         CCC DOS EST DE MT, MS E MUN DE CACOAL/RO       PDCT  IDRT  
6                CCLA SICOOB UNIMAIS METROPOLITANA       PDCT  IDRT  
7                         CCC ESTADO MG - CECREMGE       PDCT  IDRT  


In [143]:
# Tabela 3: Lista CPTECs
cptecs = buscar_dados("cptec/v1/cidade")
if cptecs:

    nome = [cptec['nome'] for cptec in cptecs]
    estado = [cptec['estado'] for cptec in cptecs]
    id = [cptec['id'] for cptec in cptecs]

    df_cptec = pd.DataFrame({
        'Nome': nome,
        'Estado': estado,
        'Id': id,
    })
    
    df_cptec['Nome'] = df_cptec['Nome'].astype(str) #Conversão de tipo de coluna
    df_cptec['Estado'] = df_cptec['Estado'].astype(str)
    df_cptec['Id'] = df_cptec['Id'].astype(int) 
    df_cptec.fillna(0, inplace=True) #Tratamento de valores ausentes
    estados_nordeste = ['AL', 'BH', 'CE', 'MA', 'PB', 'PE', 'PI', 'NA', 'SG'] # Lista de estados do Nordeste
    df_cptec_nordeste = df_cptec[df_cptec['Estado'].isin(estados_nordeste)]# Aplicar o filtro para pegar apenas os estados do Nordeste

    print(df_cptec_nordeste.head(5))

                        Nome Estado    Id
0               São Benedito     CE  4750
1  São Benedito do Rio Preto     MA  4751
2        São Benedito do Sul     PE  4752
3                  São Bento     MA  4753
4                  São Bento     PB  4754


In [144]:
#DataFrames salvos nas tabelas do banco de dados
if 'df_taxas' in locals():
    df_taxas.to_sql('Taxas', conn, if_exists='replace', index=False)

if 'df_pixs_stacked' in locals():
    df_pixs.to_sql('Pixs', conn, if_exists='replace', index=False)

if 'df_pixs_stacked' in locals():
    df_cptec_nordeste.to_sql('CPTECS', conn, if_exists='replace', index=False)

conn.commit()
conn.close()